<a href="https://colab.research.google.com/github/benjaminbrown038/Hugging-Face/blob/main/object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hugging Face Object Detection Notebook

[Imports](#imports)

[Data](#data)

##### Imports 
<a name="imports"></a>

#### Food Data 

##### Data
<a name="data"></a>